In [ ]:

import math
from typing import Tuple

import torch
import triton
import triton.language as tl

ERROR: Could not find a version that satisfies the requirement triton (from versions: none)
ERROR: No matching distribution found for triton

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
@triton.jit
def _rope_fused_kernel(
    x_ptr,
    cos_ptr,
    sin_ptr,
    out_ptr,
    STRIDE_ROW: tl.constexpr,
    HEAD_DIM: tl.constexpr,
    ROPE_DIM: tl.constexpr,
    ROPE_OFFSET: tl.constexpr,
    BLOCK_D: tl.constexpr,
):
    pid_row = tl.program_id(axis=0)
    pid_blk = tl.program_id(axis=1)

    offs_d = pid_blk * BLOCK_D + tl.arange(0, BLOCK_D)
    offs_elem = pid_row * STRIDE_ROW + offs_d
    
    x_ptrs = x_ptr + offs_elem
    out_ptrs = out_ptr + offs_elem

    mask_d = offs_d < HEAD_DIM
    x_val = tl.load(x_ptrs, mask=mask_d, other=0.0)

    mask_rope = offs_d >= ROPE_OFFSET
    offs_rope = offs_d - ROPE_OFFSET

    cos_ptrs = cos_ptr + tl.where(mask_rope, offs_rope, 0)
    sin_ptrs = sin_ptr + tl.where(mask_rope, offs_rope, 0)

    cos_val = tl.load(cos_ptrs, mask=mask_rope, other=1.0)
    sin_val = tl.load(sin_ptrs, mask=mask_rope, other=0.0)

    half_dim = ROPE_DIM // 2
    partner_ptrs = x_ptrs + tl.where(offs_rope < half_dim, half_dim, -half_dim)
    x_partner = tl.load(partner_ptrs, mask=mask_rope, other=0.0)

    sign = tl.where(offs_rope < half_dim, -1.0, 1.0)
    out_val = tl.where(
        mask_rope,
        x_val * cos_val + sign * x_partner * sin_val,
        x_val,
    )

    tl.store(out_ptrs, out_val, mask=mask_d)

In [ ]:
@triton.jit
def _rope_fused_kernel(
    x_ptr,
    cos_ptr, sin_ptr,
    out_ptr,
    STRIDE_ROW: tl.constexpr,
    HEAD_DIM : tl.constexpr,
    ROPE_DIM : tl.constexpr,
    ROPE_OFFSET : tl.constexpr,
    BLOCK_D  : tl.constexpr,
):
    pid_row = tl.program_id(axis=0)
    pid_blk = tl.program_id(axis=1)

    offs_d = pid_blk * BLOCK_D + tl.arange(0, BLOCK_D)
    offs_elem = pid_row * STRIDE_ROW + offs_d
    in_bounds = offs_d < HEAD_DIM

    x = tl.load(x_ptr + offs_elem, mask=in_bounds)

    rope_idx = offs_d - ROPE_OFFSET
    use_rope = rope_idx >= 0

    cos = tl.load(cos_ptr + tl.where(use_rope, rope_idx, 0), mask=use_rope, other=1.0)
    sin = tl.load(sin_ptr + tl.where(use_rope, rope_idx, 0), mask=use_rope, other=0.0)

    half_rope_dim = ROPE_DIM // 2
    partner = tl.load(x_ptr + offs_elem + tl.where(rope_idx < half_rope_dim, half_rope_dim, -half_rope_dim), mask=use_rope, other=0.0)
    sign = tl.where(rope_idx < half_rope_dim, -1.0,  1.0)
    out = tl.where(use_rope, x * cos + sign * partner * sin, x)

    tl.store(out_ptr + offs_elem, out, mask=in_bounds)

In [ ]:
@triton.jit
def _rope_fused_kernel(
    x_ptr, cos_ptr, sin_ptr, out_ptr,
    STRIDE_ROW: tl.constexpr,
    HEAD_DIM: tl.constexpr,
    ROPE_DIM: tl.constexpr,
    ROPE_OFFSET: tl.constexpr,
    BLOCK_D: tl.constexpr,
):
    pid_m = tl.program_id(axis=0)
    pid_d = tl.program_id(axis=1)

    offs_d = pid_d * BLOCK_D + tl.arange(0, BLOCK_D)
    offs = pid_m * STRIDE_ROW + offs_d
    is_in_bounds = offs_d < HEAD_DIM

    x = tl.load(x_ptr + offs, mask=is_in_bounds)

    offs_rope = offs_d - ROPE_OFFSET
    use_rope = offs_rope >= 0

    cos = tl.load(cos_ptr + tl.where(use_rope, offs_rope, 0),mask=use_rope, other=1.0)
    sin = tl.load(sin_ptr + tl.where(use_rope, offs_rope, 0),mask=use_rope, other=0.0)

    half_rope_dim = ROPE_DIM // 2
    rope_partner = tl.load(x_ptr + offs + tl.where(offs_rope < half_rope_dim, half_rope_dim, -half_rope_dim), mask=use_rope, other=0.0)
    sign = tl.where(offs_rope < half_rope_dim, -1.0,  1.0)
    out = tl.where(use_rope, x * cos + sign * rope_partner * sin, x)

    tl.store(out_ptr + offs, out, mask=is_in_bounds)